<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import random

## Загрузка данных

Загрузим данные:

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv') 

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Переименуем столбцы

In [5]:
df.columns = ['sex', 'age', 'income', 'family', 'payout']

Изменим тип данных

In [6]:
df.age = df.age.astype(int)
df.income = df.income.astype(int)

Посмотрим основные статистические характеристики

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
sex,5000.0,0.4990,0.500049,0.0,0.0,0.0,1.0,1.0
age,5000.0,30.9528,8.440807,18.0,24.0,30.0,37.0,65.0
income,5000.0,39916.3594,9900.082063,5300.0,33300.0,40200.0,46600.0,79000.0
family,5000.0,1.1942,1.091387,0.0,0.0,1.0,2.0,6.0
payout,5000.0,0.1480,0.463183,0.0,0.0,0.0,0.0,5.0


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Создадим модель, сохраним её в переменной model и обучим. Найдем её предсказания на обучающей выборке и сохраним их в переменной predictions. Вычислим по MSE и напечатаем на экране значение метрики R2

In [8]:
features = df.drop('payout', axis=1)
target = df['payout']

In [9]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w =  np.linalg.inv(X.T @ X) @ X.T @ y
        
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550308169177


In [10]:
model.w

array([ 7.92580563e-03,  3.57083050e-02, -1.70081903e-07, -1.35676627e-02])

In [11]:
model.w0

-0.9382354469755944

Выполним умножение матриц для проверки их свойств:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} E X^T y
$$

In [12]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
P_sq = np.random.random((5, 5))
P_sq.shape

(5, 5)

In [13]:
w = np.linalg.inv(X.T @ X) @ X.T @ y
w

array([-9.38235447e-01,  7.92580563e-03,  3.57083050e-02, -1.70081903e-07,
       -1.35676627e-02])

In [14]:
#1
w1 = np.linalg.inv((X @ P_sq).T @ (X @ P_sq)) @ (X @ P_sq).T @ y 
w1

array([ 0.65726404,  0.80747399,  0.36380988,  0.12040583, -1.95572996])

In [15]:
#2
w1 = np.linalg.inv(P_sq.T @ (X.T @ X) @ P_sq) @ (X @ P_sq).T @ y 
w1

array([ 0.65727413,  0.80748786,  0.36382333,  0.12040221, -1.95576366])

In [16]:
#3
w1 = np.linalg.inv(P_sq.T @ (X.T @ X) @ P_sq) @ (P_sq.T @ X.T) @ y 
w1

array([ 0.65727413,  0.80748786,  0.36382333,  0.12040221, -1.95576366])

In [17]:
#4
w1 = np.linalg.inv(P_sq) @ np.linalg.inv(X.T @ X) @ np.linalg.inv(P_sq.T) @ P_sq.T @ X.T @ y 
w1

array([ 0.65727672,  0.80749116,  0.36382608,  0.12040173, -1.95577186])

In [18]:
#5
w1 = np.linalg.inv(P_sq) @ np.linalg.inv(X.T @ X) @ X.T @ y
w1

array([ 0.65727672,  0.80749116,  0.36382607,  0.12040173, -1.95577186])

In [19]:
w1 = np.linalg.inv(P_sq) @ w
w1

array([ 0.65727672,  0.80749116,  0.36382607,  0.12040173, -1.95577186])

При подстановке $w' = ((XP)^T XP)^{-1} (XP)^T y$) получим $w' = P^{-1}w$ , где $w = (X^T X)^{-1} X^T y$ 

$w' = $w

**Ответ:** r2 = 0.424

**Обоснование:** Значение R2 неотрицательное, не равно нулю и не больше единицы. Значит, линейная регрессия работает.

## Алгоритм преобразования

**Алгоритм**
Умножим матрицу признаков на рандомную матрицу

In [20]:
A = df.values

In [21]:
A.shape

(5000, 5)

In [22]:
P = np.random.random((4, 4))
P

array([[0.89644514, 0.81072244, 0.44550993, 0.54877035],
       [0.80343005, 0.24748403, 0.22091599, 0.13617148],
       [0.00280213, 0.03087154, 0.30312909, 0.09017306],
       [0.29496088, 0.87226602, 0.57983402, 0.64982418]])

Квадратная матрица обратима:

In [23]:
np.linalg.inv(P)

array([[  4.28433134,  -2.29499781,   1.87417913,  -3.39722854],
       [-30.96808552,  26.11657245, -17.80070938,  23.14961476],
       [-11.80679972,  10.18283626,  -2.39642795,   8.16944522],
       [ 50.15923381, -43.10091886,  25.18169608, -35.28261332]])

In [24]:
E = P * np.linalg.inv(P)
E / np.linalg.inv(P)

array([[0.89644514, 0.81072244, 0.44550993, 0.54877035],
       [0.80343005, 0.24748403, 0.22091599, 0.13617148],
       [0.00280213, 0.03087154, 0.30312909, 0.09017306],
       [0.29496088, 0.87226602, 0.57983402, 0.64982418]])

Нашли единичную матрицу и доказали алгебраически свойство обратимости

In [25]:
A[:,:4].shape

(5000, 4)

In [26]:
P.shape

(4, 4)

In [27]:
A_coded = A[:,:4] * P[3:,:]
A_coded.shape

(5000, 4)

In [28]:
A_coded

array([[2.94960883e-01, 3.57629070e+01, 2.87597672e+04, 6.49824177e-01],
       [0.00000000e+00, 4.01242371e+01, 2.20336926e+04, 6.49824177e-01],
       [0.00000000e+00, 2.52957147e+01, 1.21765144e+04, 0.00000000e+00],
       ...,
       [0.00000000e+00, 1.74453205e+01, 1.96563732e+04, 1.29964835e+00],
       [2.94960883e-01, 1.91898525e+01, 1.89605724e+04, 1.94947253e+00],
       [2.94960883e-01, 2.44234487e+01, 2.35412611e+04, 6.49824177e-01]])

In [29]:
df_coded = pd.DataFrame(A_coded)
df_coded.shape

(5000, 4)

**Обоснование**

Данные зашифрованы с помощью рандомной матрицы 1 на 4

## Проверка алгоритма

Обучим модель на преобразованных признаках:

In [30]:
features = df_coded.copy()

model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550308169177


**Вывод** алгоритм реализован и проверен, проведено сравнение моделей до и после преобразования: качество не изменилось.